In [54]:
import json
from openai import OpenAI
import pandas as pd
import os
from tqdm import tqdm

OPENAI_API_KEY = '<YOUR_API_KEY>'
client = OpenAI(api_key=OPENAI_API_KEY)

batch_size = 50000 # Max limit for the Open AI Batch API

#### Load data

In [7]:
# Main data directory
data_directory = './'
# Cleaned data
data_file_name = 'df_clean.csv'

# Load data into DataFrame
pd.set_option('display.max_columns', None)
df = pd.read_csv(os.path.join(data_directory, data_file_name), lineterminator='\n')
df.head()

,regio1,heatingType,telekomTvOffer,newlyConst,balcony,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,hasKitchen,cellar,livingSpace,condition,streetPlain,lift,typeOfFlat,geo_plz,noRooms,floor,garden,regio2,description,facilities
0,Nordrhein_Westfalen,central_heating,ONE_YEAR_FREE,False,False,4.62,10.000000,840.00,1965.0,False,True,86.00,well_kept,Schüruferstraße,False,ground_floor,44269,4.0,1.000000,True,Dortmund,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...
1,Sachsen,floor_heating,ONE_YEAR_FREE,True,True,2.72,2.400000,1300.00,2019.0,False,True,83.80,first_time_use,Turnerweg,True,apartment,1097,3.0,3.000000,False,Dresden,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,..."
2,Bremen,self_contained_central_heating,ONE_YEAR_FREE,False,True,2.46,28.920365,903.00,1950.0,False,False,84.97,refurbished,Hermann-Henrich-Meier-Allee,False,apartment,28213,3.0,1.000000,False,Bremen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...
3,Bremen,central_heating,ONE_YEAR_FREE,False,True,1.89,2.400000,584.25,1959.0,False,True,60.30,well_kept,Lesumer_Heerstr.,False,ground_floor,28717,3.0,2.056523,False,Bremen,+ Komfortabler Bodenbelag: Die Wohnung ist zus...,Rollläden; Warmwasserbereiter; Kellerraum; Gas...
4,Baden_Württemberg,oil_heating,ONE_YEAR_FREE,False,False,3.77,40.000000,690.00,1970.0,True,True,53.00,well_kept,Hauptstraße,False,roof_storey,79211,2.0,2.000000,False,Emmendingen_Kreis,"Diese ansprechende, lichtdurchflutete DG-Wohnu...","Parkett, Einbauküche, kein Balkon"


In [8]:
df.describe(include="all")

,regio1,heatingType,telekomTvOffer,newlyConst,balcony,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,hasKitchen,cellar,livingSpace,condition,streetPlain,lift,typeOfFlat,geo_plz,noRooms,floor,garden,regio2,description,facilities
count,173373,173373,173373,173373,173373,173373.000000,173373.000000,173373.000000,173373.000000,173373,173373,173373.000000,173373,173373,173373,173373,173373.000000,173373.000000,173373.000000,173373,173373,173373,173373
unique,16,13,3,2,2,NaN,NaN,NaN,NaN,2,2,NaN,10,41778,2,10,NaN,NaN,NaN,2,419,145394,153883
top,Nordrhein_Westfalen,central_heating,ONE_YEAR_FREE,False,True,NaN,NaN,NaN,NaN,False,True,NaN,well_kept,Hauptstraße,False,apartment,NaN,NaN,NaN,False,Leipzig,Das Objekt ist ein hochwertig saniertes und de...,Laminat
freq,41023,109060,167986,158299,110189,NaN,NaN,NaN,NaN,110458,116350,NaN,78130,40252,128836,107548,NaN,NaN,NaN,136617,9789,199,193
mean,NaN,NaN,NaN,NaN,NaN,3.478081,28.835647,845.354593,1967.663555,NaN,NaN,72.434032,NaN,NaN,NaN,NaN,37316.098660,2.605484,2.044556,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,1.921718,15.184535,577.176287,35.928589,NaN,NaN,27.020618,NaN,NaN,NaN,NaN,27881.483794,0.907195,1.317701,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,-2.000000,1.000000,1.000000,1856.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,1057.000000,1.000000,-1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,2.090000,10.000000,484.280000,1956.000000,NaN,NaN,54.500000,NaN,NaN,NaN,NaN,9127.000000,2.000000,1.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,3.450000,40.000000,679.000000,1966.880541,NaN,NaN,68.000000,NaN,NaN,NaN,NaN,39106.000000,3.000000,2.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,4.670000,40.000000,1030.000000,1994.000000,NaN,NaN,87.500000,NaN,NaN,NaN,NaN,57319.000000,3.000000,3.000000,NaN,NaN,NaN,NaN


#### Create batches for the OpenAI Batch API

In [42]:
def create_batches(df, column, batch_size=50000):
    """ 
    Process the given column, prapare the batches for the Batch API, and finally save them in disk.
    """

    # Creating an array of json tasks
    tasks = []
    for index, row in df.iterrows():
        input = row[column]
        
        task = {
            "custom_id": f"{column}-{index}",
            "method": "POST",
            "url": "/v1/embeddings",
            "body": {
                # This is what you would have in your Embedding API call
                "model": "text-embedding-3-small",
                "input": input,
                "dimensions": 256
            }
        }
        
        tasks.append(task)
        
    num_files = len(tasks) // batch_size + 1
    for num_file in range(num_files):
        output_file = f'./batch_files/{column}_part{num_file}.jsonl'
        # make sure that the file does not exist, so don't add to an existing file
        if os.path.exists(output_file):
            os.remove(output_file)
        #write each embedding entry to a new line 
        with open(output_file, 'a', encoding='utf-8') as file:
            for task in tasks[batch_size*num_file : min(batch_size*(num_file+1),len(tasks))]:
                # Escaping ASCII is needed for correctly encoding German characters
                file.write(json.dumps(task, ensure_ascii=False) + '\n')
    
        # Sanity check, print the first 2 lines
        with open(output_file, 'r') as file:
            for line in file.readlines()[:2]:
                print(line)

In [43]:
create_batches(df, "description", batch_size)

{"custom_id": "description-0", "method": "POST", "url": "/v1/embeddings", "body": {"model": "text-embedding-3-small", "input": "Die ebenerdig zu erreichende Erdgeschosswohnung befindet sich in einem gepflegten 8-Familienhaus. Aufgrund der Hanglage bietet sich ein unverbaubarer Blick ins Grüne.", "dimensions": 256}}

{"custom_id": "description-1", "method": "POST", "url": "/v1/embeddings", "body": {"model": "text-embedding-3-small", "input": "Der Neubau entsteht im Herzen der Dresdner Neustadt.\nDas Baugrundstück befindet sich inmitten einer sehr gefragten Lage.\nNicht nur die zentrale Lage und die schnelle öffentliche\nVerkehrsanbindung durch den zu Fuß erreichbaren Bahnhof, wie auch Nahverkehrsanbindung, sondern auch die Architektur werden diesen\nNeubaukomplex zu einem weiteren Highlight am Dresdner Wohnungsmarkt machen.\nHier entstehen 2- bis 4-Raum Wohnungen mit Wohnflächen zwischen 43 m² und 124 m². Jede Wohnung verfügt über eine Terrasse oder einen Balkon, die Erdgeschosswohnunge

In [49]:
create_batches(df, "facilities", batch_size)

{"custom_id": "facilities-0", "method": "POST", "url": "/v1/embeddings", "body": {"model": "text-embedding-3-small", "input": "Die Wohnung ist mit Laminat ausgelegt. Das Badezimmer ist gefliest und verfügt über eine Wannendusche. Neue weiße Zimmertüren, ein Fliesenspiegel in der Küche und Fußleisten wurden kürzlich eingebaut.\nZur Wohnung gehört ein 10 m großer Keller. Eine Garage kann optional mitgemietet werden.", "dimensions": 256}}

{"custom_id": "facilities-1", "method": "POST", "url": "/v1/embeddings", "body": {"model": "text-embedding-3-small", "input": "* 9 m² Balkon\n* Bad mit bodengleicher Dusche, Badewanne und Fenster\n* Gäste-WC\n* Waschmaschinenanschluss im Bad und im Waschkeller\n* Abstell\n* Fußbodenheizung\n* Fliesen & Echtholzparkett\n* elektrische Rollläden\n* Videotürsprechanlage mit Farbdisplay\n* Aufzug\n* KfW-Effizienshaus 55\n* Tiefgaragenstellplatz (Miete bereits in der Gesamtmiete enthalten)\n\n~ Der Mietbeginn: ca. Anfang 2020\n~ Baustelle: Betreten verboten! 

#### Upload and process the prepared batches

In [58]:
# Upload batch files
batch_folder = './batch_files'
batch_input_files = []

files = os.listdir(batch_folder)
for file in tqdm(files, desc="Uploading files", unit="file"):
    batch_input_files.append(client.files.create(
        file=open(f'{batch_folder}/{file}', "rb"),
        purpose="batch"
    ))

Uploading files: 100%|████████████████████████████████████████████████████████████████████████████| 8/8 [03:19<00:00, 24.90s/file]


In [62]:
# Create the batch jobs
batch_file_ids= [batch_file.id for batch_file in batch_input_files] # Get the ids of the batch files
job_creations = []
for i,file_id in enumerate(batch_file_ids):
    job_creations.append(client.batches.create(
    input_file_id=file_id,
    endpoint="/v1/embeddings",
    completion_window="24h" # currently only 24h is supported 
    ))

In [63]:
# We can see here the jobs created, they start with validation
for job in job_creations:
    print(job)

Batch(id='batch_673a649633108190b847d1c38e23d7de', completion_window='24h', created_at=1731880086, endpoint='/v1/embeddings', input_file_id='file-IquT97bYps7n0ACRM4M8lE8c', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1731966486, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
Batch(id='batch_673a6496c4d48190af9de4f29643f5c9', completion_window='24h', created_at=1731880086, endpoint='/v1/embeddings', input_file_id='file-f4OJ7juxHBi1J4lhfGqDY9nK', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1731966486, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0)

#### Price Estimate

In [2]:
small_price_per_million = 0.010
large_price_per_million = 0.065
total_tokens_estimate = 173373 * 250
print("Estimated Total Token Count: ", total_tokens_estimate)
small_price_total = (total_tokens_estimate/1000000) * small_price_per_million
large_price_total = (total_tokens_estimate/1000000) * large_price_per_million

print("Estimated total price for the small model: ", small_price_total)
print("Estimated total price for the large model: ", large_price_total)

Estimated Total Token Count:  43343250
Estimated total price for the small model:  0.4334325
Estimated total price for the large model:  2.81731125
